In [1]:
# importing packages
import pandas as pd

In [2]:
# importing data for testing any one of the csv files content
df = pd.read_csv("./data/crypto_data/LTC-USD.csv",
                 names=['time', 'low', 'high', 'open', 'close', 'volume'])
print(df.head())

         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978


In [3]:
# importing data
main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f'./data/crypto_data/{ratio}.csv'
    df = pd.read_csv(dataset,
                     names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"},
              inplace=True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)  # joining the dataframes

In [4]:
# filling the missing values
main_df.fillna(method="ffill", inplace=True)    # forward filling
main_df.dropna(inplace=True)    # dropping the remaining null values
print(main_df.head(10))

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   
1528969020    6477.220215        2.731950      96.440002       95.524078   
1528969080    6480.000000        2.174240      96.470001      175.205307   
1528969140    6479.990234        0.903100      96.400002       43.652802   
1528969200    6478.660156        3.258786      96.400002        8.160000   
1528969260    6478.660156        1.970352      96.400002       20.425900   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  
time        

In [5]:
# creating the target column
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN (last 60 minutes)
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict? (next 3 minutes)
RATIO_TO_PREDICT = "LTC-USD"  # which crypto are we trying to predict?

In [6]:
def classify(current, future):
    if float(future) > float(current):
        return 1    # buy
    else:
        return 0    # sell

In [7]:
# creating the target column only for LTC-USD prediction
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))
print(main_df[[f'{RATIO_TO_PREDICT}_close', 'future', 'target']].head(10))

            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
1528969260      96.400002  96.449997       1


In [8]:
# displaying the entire dataframe
print(main_df.head())

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528968720      486.01001       26.019083     870.859985       26.856577   
1528968780      486.00000        8.449400     870.099976        1.124300   
1528968840      485.75000       26.994646     870.789978        1.749862   
1528968900      486.00000       77.355759     870.000000        1.680500   
1528968960 

In [9]:
# splitting the data into training and validation sets
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%

In [10]:
# preprocessing the data
from sklearn import preprocessing
from collections import deque
import numpy as np
import random

def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.
    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff
                                            # values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the NAs created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between -1 and 1.
    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    # balancing the data for near equal quantity of buy and sells data for training
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!

In [11]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [12]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930


In [13]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [19]:
# import packages
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [17]:
# some constant declaration
EPOCHS = 5  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"  # a unique name for the model

In [20]:
# model creation
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

tensorboard = TensorBoard(log_dir="logs/crypto_predict/{}".format(NAME))

filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))
# saves only the best ones

D:\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

Train on 77922 samples, validate on 3860 samples
Epoch 1/5
77922/77922 [==============================] - ETA: 0s - loss: 0.7195 - acc: 0.5192

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


77922/77922 [==============================] - 365s 5ms/sample - loss: 0.7195 - acc: 0.5192 - val_loss: 0.6941 - val_acc: 0.5060
Epoch 2/5
77922/77922 [==============================] - 366s 5ms/sample - loss: 0.6869 - acc: 0.5487 - val_loss: 0.6783 - val_acc: 0.5635
Epoch 3/5
77922/77922 [==============================] - 341s 4ms/sample - loss: 0.6826 - acc: 0.5602 - val_loss: 0.6765 - val_acc: 0.5702
Epoch 4/5
77922/77922 [==============================] - 311s 4ms/sample - loss: 0.6806 - acc: 0.5653 - val_loss: 0.6797 - val_acc: 0.5614
Epoch 5/5
77922/77922 [==============================] - 335s 4ms/sample - loss: 0.6796 - acc: 0.5668 - val_loss: 0.6785 - val_acc: 0.5751


In [22]:
# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

Test loss: 0.6784944341590368
Test accuracy: 0.5751295
